In [3]:
import numpy as np
import pandas as pd
from sympy import *

def step_function(x):
    return 1 if x > 0 else 0

class Perceptron:
    def __init__(self, learning_rate=0.01):
        self.classes_ = None # The unique classes labels.
        self.coef_ = None # Weights assigned to the features.
        self.intercept_= None # Bias vector.
        self.n_features_in_= None # Number of features seen during fit.
        self.learning_rate = learning_rate

        # Names of features seen during fit. Defined only when X has feature names that are all strings.
        self.feature_names_in_= None

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        self.feature_names_in_ = X.columns
        self.n_features_in_ = X.shape[1]

        # 행 개수: 입력 특성 개수
        # 열 개수: TLU 뉴런 개수
        # 열 벡터가 가중치 벡터임.
        self.coef_ = np.random.rand(self.n_features_in_, len(self.classes_))

        # 편향 벡터
        self.intercept_ = np.zeros(len(self.classes_))
        
        for i in range(X.shape[0]):
            for j in range(self.coef_.shape[1]):
                outputs = step_function(np.dot(X.iloc[i].values, self.coef_[:, j]) + self.intercept_[j])
                
                # 가중치 업데이트
                # 만약 타깃값과 예측값이 동일하다면 가중치는 업데이트되지 않는다.
                # 만약 타깃값이 양수인데 예측값은 음수가 나왔다면 (y[i, j] - outputs) 값은 양수가 되어 가중치에 양수를 더하게 됨.
                # 타깃값은 음수이고 예측값은 양수라면 그 반대로 가중치에 음수를 더하게 됨.
                self.coef_[:, j] += self.learning_rate * X.iloc[i].values * (y[i, j] - outputs)

                # 편향 업데이트
                self.intercept_[j] += self.learning_rate * (y[i, j] - outputs)
                    
    def predict(self, X):
        # 원래 수학에서는 행렬과 벡터의 덧셈은 정의되지 않음. 하지만 넘파이에서는 기능 제공함.
        # 아래와 같이 작성하면 넘파이가 자동으로 행렬의 각 행에 편향 벡터를 더해준다.
        z_score = (X @ self.coef_) + self.intercept_

        return step_function(z_score) # 활성화(activation) 함수 적용


In [4]:
# 비교 실험: 사용자 Perceptron vs scikit-learn Perceptron
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron as SkPerceptron
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# 1) 데이터 생성 (이진 분류)
X_np, y_np = make_classification(n_samples=1000, n_features=4, n_informative=3,
                                 n_redundant=0, n_classes=2, class_sep=1.0,
                                 random_state=42)

# 2) DataFrame/Series로 변환 (사용자 Perceptron은 pandas 입력을 기대)
feature_names = [f"x{i}" for i in range(X_np.shape[1])]
X_df = pd.DataFrame(X_np, columns=feature_names)
y_sr = pd.Series(y_np, name="target")

# 3) train/test 분리
X_train, X_test, y_train, y_test = train_test_split(X_df, y_sr, test_size=0.3, random_state=42, stratify=y_sr)

# 4) 스케일링 (scikit-learn과의 공정 비교를 위해 둘 다 같은 입력 사용)
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

# 5) 사용자 Perceptron 학습 준비: 원-핫 타깃으로 변환 (사용자 구현은 다중 출력 벡터를 기대)
classes = np.sort(y_train.unique())
class_to_index = {c: i for i, c in enumerate(classes)}
Y_train_onehot = np.zeros((len(y_train), len(classes)))
for idx, label in enumerate(y_train.values):
    Y_train_onehot[idx, class_to_index[label]] = 1

# 6) 사용자 Perceptron 학습
usr_clf = Perceptron(learning_rate=0.01)
usr_clf.fit(X_train_scaled, Y_train_onehot)

# 7) 사용자 Perceptron 예측 함수 (원-핫 출력 가정 -> argmax로 클래스 선택)
def predict_user(clf, X):
    z = (X.values @ clf.coef_) + clf.intercept_  # (n_samples, n_classes)
    # step_function은 스칼라에 정의되어 있음 -> 벡터에 적용하면 브로드캐스팅 이슈 있으므로 여기서 처리
    outputs = (z > 0).astype(int)
    # 결정은 z 점수의 argmax로 수행 (표준 퍼셉트론의 선형 결정)
    preds = np.argmax(z, axis=1)
    return preds, outputs

usr_pred_labels, usr_raw_outputs = predict_user(usr_clf, X_test_scaled)

# 8) scikit-learn Perceptron 학습/예측
sk_clf = SkPerceptron(alpha=0.0001, max_iter=1000, random_state=42)
sk_clf.fit(X_train_scaled, y_train)
sk_pred_labels = sk_clf.predict(X_test_scaled)

# 9) 성능 지표 계산
usr_acc = accuracy_score(y_test, usr_pred_labels)
sk_acc = accuracy_score(y_test, sk_pred_labels)

print("User Perceptron Accuracy:", usr_acc)
print("Sklearn Perceptron Accuracy:", sk_acc)

print("\nUser Confusion Matrix:\n", confusion_matrix(y_test, usr_pred_labels))
print("\nSklearn Confusion Matrix:\n", confusion_matrix(y_test, sk_pred_labels))

print("\nSklearn Classification Report:\n", classification_report(y_test, sk_pred_labels))


User Perceptron Accuracy: 0.8566666666666667
Sklearn Perceptron Accuracy: 0.9033333333333333

User Confusion Matrix:
 [[120  30]
 [ 13 137]]

Sklearn Confusion Matrix:
 [[143   7]
 [ 22 128]]

Sklearn Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.95      0.91       150
           1       0.95      0.85      0.90       150

    accuracy                           0.90       300
   macro avg       0.91      0.90      0.90       300
weighted avg       0.91      0.90      0.90       300

